In [1]:
# Imports as always
import numpy as np
import sympy
import cirq
import qsimcirq
from scipy.optimize import minimize

from sqif_algorithm import SQIF

In [3]:
# Instantiate an algorithm instance.
solver = SQIF(1997)

# Generate the CVP.
B, t = solver.generate_CVP(c=1.5)

# Classical approximation to the CVP.
B, t, D, w, b_op, residual_vector, step_signs = solver.find_b_op(B, t)

# Define the Hamiltonian according to the unit hypercube search problem around the approximate solution.
H = solver.define_hamiltonian(D, residual_vector, step_signs)

In [4]:
def pretty_print_hamiltonian(H):
    string_H = str(H)
    string_H = string_H.replace('+', '\n+')
    string_H = string_H.replace('-', '\n-')
    string_H = string_H.replace('*', ' * ')
    print(string_H)
    
pretty_print_hamiltonian(H)

43.500 * I
-4.000 * Z(q(0)) * Z(q(1))
+2.500 * Z(q(0)) * Z(q(2))
-3.500 * Z(q(1))
-4.000 * Z(q(2))
+3.000 * Z(q(1)) * Z(q(2))
-1.500 * Z(q(0))


In [5]:
def generate_gamma_layer(H, i):
    """
    Generate the i-th gamma layer executing the unitary exp(-i * gamma * H).
    
    :param H: The Hamiltonian.
    :param i: Layer index.
    :return: [] of cirq.DensePauliString object corresponding to the i-th gamma layer in the QAOA circuit.
    """
    
    # Gamma symbol placeholder.
    gamma = sympy.Symbol(f'gamma_{i}')
    
    # Instantiate the DensePauliString operators.
    dense_I = cirq.DensePauliString('')
    dense_Z = cirq.DensePauliString('Z')
    dense_ZZ = cirq.DensePauliString('ZZ')
    
    # Consider the terms in the Hamiltonian.
    for term in H:
        # Split the term into its coefficient and operator.
        coefficient = term.coefficient
        operator = term.with_coefficient(1).gate
        
        # Map to the appropriate circuit element on the basis of the operator, parameterised by gamma.
        if operator == dense_ZZ:
            yield cirq.ZZ(*term.qubits) ** (gamma * coefficient)
        elif operator == dense_Z:
            yield cirq.Z(*term.qubits) ** (gamma * coefficient)
        elif operator == dense_I:
            yield []
        else:
            raise Exception(f'Unrecognised Pauli string term {term} in the Hamiltonian.')
        
        
def generate_beta_layer(qubits, i):
    """
    Generate the i-th beta layer, executing a Pauli-X raised to beta across the given qubits.
    
    :param qubits: The qubits in the circuit.
    :param i: Layer index.
    :return: [] of cirq.DensePauliString object corresponding to the i-th beta layer in the QAOA circuit.
    """
    
    # Beta symbol placeholder.
    beta = sympy.Symbol(f'beta_{i}')
    
    # The layer is trivially defined by NOT gates parameterised by beta.
    return [cirq.X(q) ** beta for q in qubits]


def generate_qaoa_circuit(H, p=1):
    """
    Generate a QAOA circuit for the given Hamiltonian with a given depth.
    
    :param H: The Hamiltonian.
    :param p: Depth of the circuit (should be kept relatively small -- say 1 to 5).
    :return: cirq.Circuit object performing QAOA with the given Hamiltonian p times (depth p).
    """
    
    # Number of qubits.
    qubits = H.qubits
    
    # Define the circuit.
    return cirq.Circuit(
        # Hadamard over all qubits first to open uniform superposition.
        cirq.H.on_each(*qubits),
        
        # p layer of QAOA-ness.
        [
            (
                # Gamma layer.
                generate_gamma_layer(H, i),
                
                # Beta layer.
                generate_beta_layer(qubits, i)
            )
            for i in range(p)
        ]
    )


circuit = generate_qaoa_circuit(H, p=1)
circuit

┌────────────────────────────────┐
0: ───H───ZZ───────────────────ZZ──────────────────────────────────Z^(-1.5*gamma_0)───X^beta_0──────────────────────
          │                    │
1: ───H───ZZ^(-4.0*gamma_0)────┼───────────────Z^(-3.5*gamma_0)───────────────────────ZZ─────────────────X^beta_0───
                               │                                                      │
2: ───H────────────────────────ZZ^(2.5*gamma_0)────────────────────Z^(-4.0*gamma_0)───ZZ^(3.0*gamma_0)───X^beta_0───
                              └────────────────────────────────┘

In [6]:
def find_optimal_parameters(circuit, H):
    """
    Find the optimal assignments over the parameters in the given circuit.
    
    :param circuit: The QAOA circuit.
    :param H: The Hamiltonian.
    :return: cirq.ParamResolver object assigning values to each placeholder parameter in the given circuit.
    """
    
    # Define the parameters and observables.
    parameters = sorted(cirq.parameter_names(circuit))
    observables = [term.with_coefficient(1) for term in H]
    
    # Define the function to be minimised -- the expectation value with given assignments.
    def func_to_minimise(x):
        # Assign the parameters their given values.
        parameter_assignments = cirq.ParamResolver({param : val for param, val in zip(parameters, x)})
        
        # Simulator object.
        simulator = qsimcirq.QSimSimulator(
            qsimcirq.QSimOptions(cpu_threads=8, verbosity=0)
        )
        
        # Simulate the expectation value.
        result = simulator.simulate_expectation_values(
            program=circuit, observables=observables, param_resolver=parameter_assignments
        )
        
        # Compute the return, ignoring the imaginary component.
        return sum(term.coefficient * val for term, val in zip(H, result)).real
    
    # Initialise the assignments (with no prior knowledge, all zeros is fine).
    x0 = np.asarray([0.0] * len(parameters))
    
    # Minimise the function over the parameters.
    result = minimize(func_to_minimise, x0, method='Nelder-Mead')
    return cirq.ParamResolver({param: optimal_val for param, optimal_val in zip(parameters, result.x)})
        

optimal_parameters = find_optimal_parameters(circuit, H)
optimal_parameters

cirq.ParamResolver({'beta_0': 0.3205382149444925, 'gamma_0': -0.06605087626914322})

In [7]:
def sample_bitstring_from_parameters(circuit, parameter_assignments, H, repetitions):
    """
    Sample the states obtained via measurements made on the circuit with the given parameter assignments, given as a histogram of repeated runs.
    
    :param circuit: The circuit to measure.
    :param parameter_assignments: The assignments to the parameters (as a cirq.ParamResolver object).
    :param H: The Hamiltonian (whose qubits are operated on).
    :param repetitions: Number of times to repeat the sampling.
    :return: Histogram of states sampled by the measurements.
    """
    
    # Simulator object.
    simulator = qsimcirq.QSimSimulator(
        qsimcirq.QSimOptions(cpu_threads=8, verbosity=0)
    )
    
    # Add a set of measurement operators to the circuit.
    measurement_circuit = circuit + cirq.Circuit(cirq.measure(H.qubits, key='m'))
    
    # Run the simulation.
    result = simulator.run(measurement_circuit, param_resolver=parameter_assignments, repetitions=repetitions)
    
    # Let's have a histogram.
    return result.histogram(key='m')


n_samples = 10000
states_histogram = sample_bitstring_from_parameters(circuit, optimal_parameters, H, n_samples)
outcomes, frequencies = zip(*states_histogram.most_common(len(states_histogram)))

for x, y in zip(outcomes, frequencies):
    print(f'{x}, {y}')

6, 3281
0, 2765
1, 2595
4, 972
2, 253
3, 51
7, 42
5, 41


In [8]:
def integer_outcomes_to_lattice_vectors(states, w, D, step_signs):
    # Convert the integer state to a binary state -- telling us WHETHER to step in each basis direction.
    m = 3 # This is an attribute of the class, so we'll hardcode it here.
    binary_states = (((states[:, None] & (1 << np.arange(m)[::-1]))) > 0).astype(int)
    
    # Pairwise-multiply the binary states and step signs -- telling us HOW to step in each basis.
    steps = np.multiply(binary_states, step_signs)
    
    # Add the steps to the Babai weights.
    w_new = w + steps
    
    # Left-multiply to yield the new vector v_new on the lattice corresponding to the states.
    return w_new @ D.T

lattice_vectors = integer_outcomes_to_lattice_vectors(np.array(outcomes), w, D, step_signs)
lattice_vectors

array([[  3,   5,   0, 241],
       [  0,   4,   4, 242],
       [  3,   2,   4, 238],
       [ -1,   6,   2, 239],
       [  4,   3,   2, 244],
       [  7,   1,   2, 240],
       [  6,   3,   0, 237],
       [  2,   4,   2, 235]])

In [9]:
print(f'Reminder: the residual vector b_op - t = {residual_vector} gives distance to target {np.linalg.norm(residual_vector)}.')
print(f'This came from b_op = {b_op} and t = {t}\n')

for v_new, freq in zip(lattice_vectors, frequencies):
    print(f'With prob {round(freq / n_samples, 3):.3f}, v_new = {v_new} with distance to target {np.linalg.norm(v_new - t)}.')

Reminder: the residual vector b_op - t = [0 4 4 2] gives distance to target 6.0.
This came from b_op = [  0   4   4 242] and t = [  0   0   0 240]

With prob 0.328, v_new = [  3   5   0 241] with distance to target 5.916079783099616.
With prob 0.277, v_new = [  0   4   4 242] with distance to target 6.0.
With prob 0.260, v_new = [  3   2   4 238] with distance to target 5.744562646538029.
With prob 0.097, v_new = [ -1   6   2 239] with distance to target 6.48074069840786.
With prob 0.025, v_new = [  4   3   2 244] with distance to target 6.708203932499369.
With prob 0.005, v_new = [  7   1   2 240] with distance to target 7.3484692283495345.
With prob 0.004, v_new = [  6   3   0 237] with distance to target 7.3484692283495345.
With prob 0.004, v_new = [  2   4   2 235] with distance to target 7.0.
